<a href="https://colab.research.google.com/github/alexik777/alexik777/blob/main/Transform_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# В этом блокноте мы доучим готового трансформера на основе отзывов автовладельцев auto.ru генерировать текст.

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 38.2 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 6.5 MB 36.1 MB/s 
     |████████████████████████████████| 895 kB 41.1 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import re
import random
import glob 

import torch
from tqdm.notebook import tqdm
import transformers
from torch.optim import AdamW

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Загружаем токенайзер модели
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [ ]:
! unzip /content/drive/MyDrive/Datasets/reviews.zip

Streaming output truncated to the last 5000 lines.
  inflating: review_1157.txt         
  inflating: review_1158.txt         
  inflating: review_1159.txt         
  inflating: review_1160.txt         
  inflating: review_1161.txt         
  inflating: review_1162.txt         
  inflating: review_1163.txt         
  inflating: review_1164.txt         
  inflating: review_1165.txt         
  inflating: review_1166.txt         
  inflating: review_1167.txt         
  inflating: review_1168.txt         
  inflating: review_1169.txt         
  inflating: review_1170.txt         
  inflating: review_1171.txt         
  inflating: review_1172.txt         
  inflating: review_1173.txt         
  inflating: review_1174.txt         
  inflating: review_1175.txt         
  inflating: review_1176.txt         
  inflating: review_1177.txt         
  inflating: review_1178.txt         
  inflating: review_1179.txt         
  inflating: review_1180.txt         
  inflating: review_1181.txt         

In [ ]:
# Загружаем файлы датасета с обзорами автомобилей, чистим, конкатенируем, готовим для дообучения модель
text = ''
for filename in glob.glob('*.txt'):
  with open(filename, encoding='cp1251') as f:
    text_piece = f.read()
# text = re.split(r'\n', text,maxsplit=1)
    text_piece = re.sub('\d+\n', ' ', text_piece)
    text_piece = re.sub('\n', ' ', text_piece)
    text_piece = re.sub(' +', ' ', text_piece)
    text = text + text_piece+'\n'

print(text[:300])

Great Wall Hover - 2008 - «Мой любимчик» Купили этот автомобиль в декабре 2008 года и жил он в нашей семье до 2013. Считаю, что покупать новый автомобиль нужно через 5-6 лет, так как дальше начинаются всякие там поломочки и это уже не комильфо. Что могу сказать - машина зверь, переднеприводная с под


In [ ]:
# токенизируем текст
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

2355422


array([   43,  5879, 27971,  1021,  4054,   376,  5049,   376,   478,
       22012])

In [ ]:
# разбиваем на train и test

l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

2355422 1884336 471084


In [ ]:
from transformers import GPT2LMHeadModel

# Эту модель подгрузим и далее обучим
model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
batch_size = 8
max_len = 256
epochs = 5

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

# трансформеры с трудом обучаются, для них нужны разные способы повышения 
# эффективности градиентного спуска
total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

920 230


In [ ]:
# готовим тензоры для обучения

def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


# обучающий цикл
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None, 
                                
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
        b    loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/5 : training


  0%|          | 0/920 [00:00<?, ?it/s]

epoch 1/5 : validation


  0%|          | 0/230 [00:00<?, ?it/s]

epoch 2/5 : training


...

  0%|          | 0/230 [00:00<?, ?it/s]

epoch 5/5 : training


  0%|          | 0/920 [00:00<?, ?it/s]

epoch 5/5 : validation


  0%|          | 0/230 [00:00<?, ?it/s]

In [ ]:
import textwrap

In [ ]:
prompt = 'Зеркало заднего вида черное'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=5,
    do_sample=True,
    temperature=10.,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Зеркало заднего вида черное.В итоге я приобрела авто с зеркальным передним стеклом - оно не очень плохое в обслуживании
для нашего климата за год использования. В дороге вожусь достаточно спокойно - в салоне комфортно, и шум в машине просто
не чувствуется, что там едет машина. За рулем автомобиля меня можно сказать даже очень хорошо :), и если на дороге у Вас
в городе не особо хорошо слышно как едет Ваш друг (я сама часто пользуюсь данным транспортном средством - и муж не
против), но у нас на парковке - тихо и спокойно (даже муж с дачи не кричит при движении, пока едет в машину). За городом
я не выхожу из авто, только паркуясь (это вообще не страшно). Если вы хотите в своем автомобиле ездить
------------------
Зеркало заднего вида черное, а по бокам очень сильно прорезана дырами (если вы живете рядом на дачных, огородиках)! Я бы
такую на себе не смогла с собой тащить! В салоне, кроме меня (и водителя с пассажиром) было 2 мужика - муж и папа. Они
постоянно сидели сзади сзади) А теперь п